In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Define image size, batch size, and initial learning rate
IMG_SIZE = 224
BATCH_SIZE = 32
INITIAL_EPOCHS = 2000
FINE_TUNE_EPOCHS = 200
INITIAL_LEARNING_RATE = 1e-3

# Data augmentation to improve generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the images from the directory and set the number of classes automatically
train_generator = train_datagen.flow_from_directory(
    'path_to_train_images/archive/imagenet-sketch/sketch', 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

# Get the number of classes from the data generator
number_of_classes = train_generator.num_classes

# Load and freeze base layers
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model initially

# Create the model with dropout and L2 regularization
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  # L2 regularization
    Dropout(0.5),  # Dropout for regularization
    Dense(number_of_classes, activation='softmax')  # Output layer for classification
])

# Compile the model with the initial learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

# Train the model with initial frozen layers
history = model.fit(train_generator, epochs=INITIAL_EPOCHS, callbacks=[early_stopping])

# Fine-tune the base model by unfreezing some layers
base_model.trainable = True
for layer in base_model.layers[:100]:  # Freeze only the first 100 layers
    layer.trainable = False

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(train_generator, epochs=FINE_TUNE_EPOCHS, callbacks=[early_stopping])

# Convert to TFLite model with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]  # Quantize to float16 for mobile optimization
tflite_model = converter.convert()

# Save the quantized TFLite model
with open('optimized_image_recognition_model.tflite', 'wb') as f:
    f.write(tflite_model)